#1. Iris 데이터 얻기

#2. k-means 군집화

#3. 실루엣 계수 확인 및 추가

#4. 평균 실루엣 계수 구하기

#5. 시각화

> 위의 그래프 결과 클러스터가 2개일 경우 가장 높은 실루엣 점수를 갖게 되며 다음은 클러스터가 4개인 경우이다.

> iris의 경우 2개의 클러스터인 경우가 가장 높고 다음 순차적으로 스코어 점수가 작아지는 것을 볼 수 있다.